In [ ]:
#default_exp utils
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#export
from nbdev.showdoc import *

# Utils
> Functions to help the inner workings of `gingado`

In [ ]:
#hide
#export
import datetime
import os
import pwd

In [ ]:
#hide
#export
def get_username():
    "Returns the active username in the computer"
    return pwd.getpwuid(os.getuid()).pw_name

In [ ]:
show_doc(get_username)

<h4 id="get_username" class="doc_header"><code>get_username</code><a href="__main__.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>get_username</code>()

Returns the active username in the computer

In [ ]:
u = get_username()
assert isinstance(u, str)
assert len(u) > 0

In [ ]:
#hide
#export
def get_datetime():
    "Returns the time now"
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S %Z") 

In [ ]:
show_doc(get_datetime)

In [ ]:
d = get_datetime()
assert isinstance(d, str)